---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [343]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [194]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df=pd.read_table('university_towns.txt' , header=None) # readign the data set , u can also read it as csv and put the seperator as /n
    
    def func1(df):
        #creating the lists to put the names in it for each state and region
        st=[]
        reg=[]
        for i in range (len(df) ) : # iterating through the rows of the df 
            if '[edit]' in df.loc[i]: # if the word [edit] is present in the row
                st.append(df.loc[i].split('[')[0]) # split the words on the sympol [  into 2 words and take the first word and append it to the state list as i_th item
                reg.append(np.NaN) # append nan value to the same i_th item in the region list , to remove that row later 
                
            else: # if there is no [edit] then this is a region name and not a state
                reg.append( df.loc[i].split(' (')[0] ) # split the ( of the name of the region if found , and append the first word to the region list as i_th item
                st.append(st[i-1]) # append the same name of the state from the i_th step before, we cannot use nan values because it will make issues when cleaning the data
                
        return st , reg #pd.DataFrame([st , reg],index=['st','reg'])
    
    df= df.apply(func1) # applying the function
    
    df= pd.DataFrame([df[0][0],df[0][1]] , index=['State','RegionName'] ).T  # creating the data frame from the state and region lists and giving them the right indexing and transposing the df

    df = df.dropna() # dropping the rows with NaN values in it, we could do that in the step above also
 
    return df
get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
...,...,...
561,Wisconsin,River Falls
562,Wisconsin,Stevens Point
563,Wisconsin,Waukesha
564,Wisconsin,Whitewater


In [294]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls',skiprows=219).dropna(axis=1).rename(columns={'1999q4':'Quarter' ,9926.1:'drop' , 12323.3 : 'GDP' }).T
    df=df.drop('drop').T
    df=df.set_index('Quarter')
    scale=0
    for i in range (len(df) -1 ):    
        if ( df.iloc[i,0] > df.iloc[i+1,0]) & (df.iloc[i+1 , 0] > df.iloc[i+2 , 0]) : 
            # making condition as the i+1_th q is the start of the recession when the it's less than the previous q and bigger than the next q
            
            return df[( df== df.iloc[i+1,0] ).values].index[0]
        
    df[( df== df.iloc[i,0] ).values].index[0]
    return 
get_recession_start()

'2008q3'

In [305]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls',skiprows=219).dropna(axis=1).rename(columns={'1999q4':'Quarter' ,9926.1:'drop' , 12323.3 : 'GDP' }).T
    df=df.drop('drop').T
    df=df.set_index('Quarter')
    df= df.loc['2008q3':]
    for i in range (len(df) -1 ):    
         if ( df.iloc[i,0] < df.iloc[i+1,0]) & (df.iloc[i+1 , 0] < df.iloc[i+2 , 0]) : 
            # making condition as the i+2_th q is the end of the recession when the it's bigger than the previous q and the previous i+1_th q is bigger than it's previous i_th q
            
            return df[( df== df.iloc[i+2,0] ).values].index[0]
        

    return 
get_recession_end()

'2009q4'

In [338]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df= pd.read_excel('gdplev.xls',skiprows=219).dropna(axis=1).rename(columns={'1999q4':'Quarter' ,9926.1:'drop' , 12323.3 : 'GDP' }).T
    df= df.drop('drop').T
    df= df.set_index('Quarter')
    df= df.loc['2008q3':'2009q4']
    #df.iloc[i+1,0]
    
    return df[( df.loc[:,'GDP']  == (df.min().values[0]) )].index[0]
get_recession_bottom()

'2009q2'

In [506]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    
    
    # column names to drop later, copy paste from excel and divide them into  list of string to be able to drop them 
    y_drop='RegionID	Metro	CountyName	SizeRank 1996-04 1996-05 1996-06	1996-07	1996-08	1996-09	1996-10	1996-11	1996-12	1997-01	1997-02	1997-03	1997-04	1997-05	1997-06	1997-07	1997-08	1997-09	1997-10	1997-11	1997-12	1998-01	1998-02	1998-03	1998-04	1998-05	1998-06	1998-07	1998-08	1998-09	1998-10	1998-11	1998-12	1999-01	1999-02	1999-03	1999-04	1999-05	1999-06	1999-07	1999-08	1999-09	1999-10	1999-11	1999-12'
    y_drop=y_drop.split()
    years_dic='2000-01	2000-02	2000-03	2000-04	2000-05	2000-06	2000-07	2000-08	2000-09	2000-10	2000-11	2000-12	2001-01	2001-02	2001-03	2001-04	2001-05	2001-06	2001-07	2001-08	2001-09	2001-10	2001-11	2001-12	2002-01	2002-02	2002-03	2002-04	2002-05	2002-06	2002-07	2002-08	2002-09	2002-10	2002-11	2002-12	2003-01	2003-02	2003-03	2003-04	2003-05	2003-06	2003-07	2003-08	2003-09	2003-10	2003-11	2003-12	2004-01	2004-02	2004-03	2004-04	2004-05	2004-06	2004-07	2004-08	2004-09	2004-10	2004-11	2004-12	2005-01	2005-02	2005-03	2005-04	2005-05	2005-06	2005-07	2005-08	2005-09	2005-10	2005-11	2005-12	2006-01	2006-02	2006-03	2006-04	2006-05	2006-06	2006-07	2006-08	2006-09	2006-10	2006-11	2006-12	2007-01	2007-02	2007-03	2007-04	2007-05	2007-06	2007-07	2007-08	2007-09	2007-10	2007-11	2007-12	2008-01	2008-02	2008-03	2008-04	2008-05	2008-06	2008-07	2008-08	2008-09	2008-10	2008-11	2008-12	2009-01	2009-02	2009-03	2009-04	2009-05	2009-06	2009-07	2009-08	2009-09	2009-10	2009-11	2009-12	2010-01	2010-02	2010-03	2010-04	2010-05	2010-06	2010-07	2010-08	2010-09	2010-10	2010-11	2010-12	2011-01	2011-02	2011-03	2011-04	2011-05	2011-06	2011-07	2011-08	2011-09	2011-10	2011-11	2011-12	2012-01	2012-02	2012-03	2012-04	2012-05	2012-06	2012-07	2012-08	2012-09	2012-10	2012-11	2012-12	2013-01	2013-02	2013-03	2013-04	2013-05	2013-06	2013-07	2013-08	2013-09	2013-10	2013-11	2013-12	2014-01	2014-02	2014-03	2014-04	2014-05	2014-06	2014-07	2014-08	2014-09	2014-10	2014-11	2014-12	2015-01	2015-02	2015-03	2015-04	2015-05	2015-06	2015-07	2015-08	2015-09	2015-10	2015-11	2015-12	2016-01	2016-02	2016-03	2016-04	2016-05	2016-06	2016-07	2016-08'
    years_dic=years_dic.split()

    
    df = pd.read_csv('City_Zhvi_AllHomes.csv').drop(y_drop ,axis=1).fillna(0) # filling the nan values with 0 to not interfere with np.mean
    df['State'] = df['State'].apply( lambda x: states[x] ) # using lambda function to change the values of the rows of the state using the states dictionary
    df=df.set_index(["State","RegionName"])
    
    x=1 # setting the quarter indicator starting from 1:4
    c=2000 # setting the year variable
    
    for i in range( int(len(df.columns) /3) ): #since we are going to divide each year into 3 months section we are dividing the cols number by 3
        start= 3*i # setting the starting point 
        end=start+3 # setting the end point with a step of 3
        
        df[ '{}q{}'.format( c , x ) ]= np.mean( df.iloc[:, start : end ] , axis=1)  # creating a new col with the  year and quarter number and putting the mean values of the 3 col in it
        if x == 4: # every 4 quarters reset the quarter value and increase year 
            x=1
            c+=1
        else: # every iteration increase x
            x+=1

    df[ '2016q3' ]= np.mean( df.iloc[:, 3*66 : 3*66+2 ] , axis=1) # creating the last col as it only contains 2 months not 3 , and it was making bugs in the code 
    
    df=df.drop(years_dic,axis=1) # dropping the years-months cols
    
    
    
    
    return df 
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [707]:
def ttesting():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Returns the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p is equal to
    the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
        
    start= get_recession_start() # the col name of the recession start
    end= get_recession_end() # col name of recession end
    bottom= get_recession_bottom() # col name of the least gdp quarter 
    
    
    prices= convert_housing_data_to_quarters() # converts the yearly prices into quarters
    unitown= get_list_of_university_towns() # state , region name
    
    df= prices.loc[:,[start,bottom]]
    
    df['trend']= df[start]-df[bottom] # decline or growth of housing prices
    
    # univercity prices:
    uniprices= pd.merge(df , unitown , on=['State', 'RegionName'] , how='inner',  right_index=True).dropna()['trend']
    
    nonuni= df.drop(uniprices.index)['trend'].dropna() # taking only the data of the whole-univ and taking the col 'trend'
    
    p = ttest_ind(uniprices , nonuni ).pvalue
 
    if p < 0.1 : difference = True
    elif p>=0.1 : difference = False
        
    # getting the mean of the trend for each data
    un=np.mean(uniprices) 
    no=np.mean(nonuni)
    # set better depending on the better/lower price value
    better= 'university town' if un < no else 'non university town'
    
    return   difference,p,better #the right answer should be: True, 7.1245161701733e-06, 'university town'
ttesting()

(True, 0.08142112201413776, 'university town')